In [1]:
import numpy as np 
from kmcluster.core.utils import fetch_huanchen_plot
from kmcluster.core.data import sparse_to_mat
from kmcluster.core.data import energy_to_rates
from kmcluster.core.intialize import random_init, boltz
from kmcluster.core.viz import graph_trajectories_static
from kmcluster.core.cluster import (
    plot_coms_cdlib,
    plot_affinity_at_temp,
)

# supress warnings
import warnings

warnings.filterwarnings("ignore")

/home/santiagovargas/anaconda3/envs/allegro/lib/python3.9/site-packages/brisk/funcs.py:99: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dot(x, y):
/home/santiagovargas/anaconda3/envs/allegro/lib/python3.9/site-packages/brisk/funcs.py:174: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def interp(x, xp, fp):
/home/santiagovargas/anaconda3/envs/allegro/lib/pyt

In [2]:
# get data
T = 100 # temperature
huan_all, huan_rel = huan_bars = fetch_huanchen_plot()
# generate boltzman distribution
boltz_dist = boltz(size=10000, energies=huan_rel, T=T)
energies_huan_mat = sparse_to_mat(huan_all)

In [3]:
#energies_relative = energies_huan_mat - np.max(energies_huan_mat)
rate_mat = energy_to_rates(energies_huan_mat, T, scale=1)
# convert rate matrix to transition matrix
transition_mat = rate_mat / np.sum(rate_mat, axis=1)[:, np.newaxis]
# convert rate matrix to rate Matrix where diagonal is -1 * sum of column
rate_mat_diag_neg = rate_mat - np.diag(np.sum(rate_mat, axis=0))

In [4]:
# get eigenvalues and eigenvectors of transition_mat 
eigvals, eigvecs = np.linalg.eig(transition_mat)
# find stationary distribution
pi = eigvecs[:, np.argmax(eigvals.real)]
# normalize stationary distribution
pi = pi / np.sum(pi)
# filter out states with low probability
pi[pi < 1e-3] = 0

# this is a t --> inifinity
print("Stationary distribution: ", pi.real)
print("States visited by stationary distribution: ", np.nonzero(pi)[0])
# this is zero indexed !!!

Stationary distribution:  [0.         0.05650384 0.         0.05650384 0.05650383 0.
 0.05650383 0.05650384 0.05409612 0.05409612 0.0540964  0.05409612
 0.         0.05409612 0.         0.         0.05409637 0.05650383
 0.05645581 0.05650622 0.05409612 0.05388411 0.05650384 0.
 0.         0.05650384 0.0540964  0.         0.05650384 0.        ]
States visited by stationary distribution:  [ 1  3  4  6  7  8  9 10 11 13 16 17 18 19 20 21 22 25 26 28]


In [5]:
# get eigenvalues and eigenvectors of rate matrix
eigvals, eigvecs = np.linalg.eig(rate_mat_diag_neg)
# sort eigenvalues and eigenvectors
idx = eigvals.argsort()[::-1]
# make any eigenvalues close to zero zero
#eigvals[eigvals] = 0
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

eig_mat = np.diag(eigvals)
eig_vec_mat = eigvecs

eig_vec_mat_inv = np.linalg.inv(eig_vec_mat)
# if this give you an singular matrix error use np.linalg.pinv instead

def taylor_expand_prop(eig_mat, t, order = 3):
    # initialize matrix
    taylor_mat = np.zeros_like(eig_mat)
    # loop over order
    for i in range(order):
        # add to matrix
        taylor_mat += np.linalg.matrix_power(t * eig_mat, i) / np.math.factorial(i)
    return taylor_mat

def propagate(a_0, time, eig_mat, eigvecs, inv_eigvecs, order = 3):
    """
    a_0: initial distribution
    time: time to propagate
    eigvecs: eigenvectors of rate matrix
    inv_eigvecs: inverse of eigenvectors of rate matrix
    order: order of taylor expansion
    """
    # initialize matrix
    prop_mat = np.zeros_like(eigvecs)
    # loop over order
    taylor_mat = taylor_expand_prop(eig_mat, time, order)
    # multiply by eigenvectors
    prop_mat = np.matmul(eigvecs, np.matmul(taylor_mat, inv_eigvecs))
    # multiply by initial distribution
    prop_mat = np.matmul(prop_mat, a_0)
    return prop_mat

def propagate_alt(a_0, time, eig_mat, eigvecs, inv_eigvecs, order = 3):
    """
    a_0: initial distribution
    time: time to propagate
    eigvecs: eigenvectors of rate matrix
    inv_eigvecs: inverse of eigenvectors of rate matrix
    order: order of taylor expansion
    """
    ret_mat = np.zeros_like(a_0, dtype=np.complex128)
    # loop over all eigenvectors
    for i in range(len(eigvecs)):
        # get eigenvalue
        eig_val = eig_mat[i, i]
        # get eigenvector
        eig_vec = eigvecs[:, i]
        # get inverse eigenvector
        inv_eig_vec = inv_eigvecs[i, :]
        # get taylor expansion
        #print(inv_eig_vec * a_0 * np.exp(-1 * eig_val * time) * eig_vec)
        ret_mat += inv_eig_vec * a_0 * np.exp(-1 * eig_val * time) * eig_vec
    
    return ret_mat

In [6]:
boltz_dist = boltz(size=10000, energies=huan_rel, T=T)
init_dist = boltz_dist.get_init_populations()
init_dist_prop = init_dist / np.sum(init_dist)

In [13]:
new_state = propagate(init_dist_prop, 10e-6, eig_mat, eigvecs, eig_vec_mat_inv, order=2)
# print without complex
print(new_state.real * 10000)
print(np.sum(new_state.real)) # should be 1


[-6.18596616e+09 -8.78480516e+08  2.51174914e+09  8.61236639e+08
 -1.48069690e+09  3.62412014e+09  4.38106447e+08 -2.29045567e+06
 -3.63914601e+09  2.37378588e+05  3.41934965e+02  3.63890948e+09
 -1.49020329e+06 -1.22275267e+08 -1.14381208e+07 -4.08482716e+08
  3.36998769e+02  1.04259141e+09  3.56999996e+02  3.78993956e+02
  1.22276061e+08  3.37000000e+02  1.72448712e+07  4.58581170e+08
  3.45930575e+02  2.29112790e+06  3.44200915e+02  1.29293441e+07
  3.41083586e+02  3.58327879e+02]
1.0000041027029531


In [12]:
init_dist

[311,
 315,
 311,
 340,
 292,
 304,
 340,
 354,
 342,
 336,
 339,
 325,
 358,
 333,
 321,
 325,
 337,
 326,
 357,
 379,
 312,
 337,
 340,
 321,
 344,
 318,
 344,
 341,
 341,
 357]